In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

num_points = 20
radius = 8.00

1 sample 
distance double
all the node tranmits
start with 50 scenario

In [2]:
import os
# Play a notification sound
notification_sound = "program_completed.wav"  # Replace with the actual audio file name
# os.system(f"afplay {notification_sound}")

In [3]:
# generating random x and y coordinates of the tags with values between-15 to 15
# x_coords = np.random.uniform(-15, 15, 20)
# y_coords = np.random.uniform(-15, 15, 20)

#0
x_coords = [ -5.32463153,  -8.76051415,  -1.15636462,   2.19195406,   3.21037006,
  14.11990191, -14.26377168, -12.93228116,   2.58042298,   5.93712745,
   8.08366368,   9.57134564,  -2.85396558, -13.11097522,  -7.59629256,
  -4.24879346,  -3.31284688,  -9.70391303,   6.36873875,  10.41085005]
y_coords = [-14.2935121,    5.67973811,   4.08538733,  -3.04254408,   4.14224576,
   8.74004094, -10.90073777,  -2.88978037,  -9.26896965, -14.57394033,
  -4.0107698,   13.50963778,  -2.72081589,   4.94863212,   3.07360091,
 -12.12926027,   8.99771014, -13.81522331,   4.15886645,   9.65946393]
point_labels = [f"P{i + 1}" for i in range(num_points)]

In [4]:
def saveplot(name):
    
    
    subdirectory = "experiments"
    path = os.path.join(os.getcwd(), subdirectory)
    filepath = os.path.join(path, name)
    
    
    
    # for Figure plotting
    fig, ax = plt.subplots()
    # Set the x and y limits of the axes
    ax.set_xlim([-20, 20])
    ax.set_ylim([-20, 20])

    # Plot the points on the axis
    # ax.scatter(x_coords, y_coords)
    for i in range(num_points):
        ax.scatter(x_coords[i], y_coords[i])
        ax.annotate(point_labels[i], (x_coords[i], y_coords[i]))

    # Add gridlines
    ax.grid(True)

    # Set the title and axis labels
    ax.set_title(f"Random Points on Cartesian Plane ({num_points} points)")
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")
    
    
    
    plt.savefig(filepath)

In [5]:
import pandas as pd
import numpy as np
def savedata(data, name):
    
    subdirectory = "experiments"
    path = os.path.join(os.getcwd(), subdirectory)
    filepath = os.path.join(path, name)
    
    df = pd.DataFrame(data)
    df.columns = df.columns + 1

    # Save the DataFrame to an Excel file
    df.to_excel(filepath, index=False)
    

In [6]:
# import random

# # function to select nodes requesting to transmit their messages.
# def select_nodes():
#     nodes = [random.uniform(0,1) for _ in range(num_points + 1)]  
#     nodes = [0 if v < 0.4 else 1 for v in nodes]
#     nodes[0] = 0  #X[0] is default 0 to make 1-based indexing
#     return nodes

import math
import random

# function to select nodes requesting to transmit their messages.
def select_nodes():
    num_of_nodes = math.ceil(random.uniform(0,1) * num_points)
    indexes = list(range(1, num_points + 1))
    #print(indexes)
    
    #print(num_of_nodes)
    selected_indexes = random.sample(indexes, num_of_nodes)
    
    #print(selected_indexes)
    nodes = [0 for i in range (num_points + 1)]
    
    for num in selected_indexes:
        nodes[num] = 1
    #print(nodes)
    return nodes

# import random

# # function to select nodes requesting to transmit their messages.
# def select_nodes():
#     nodes = [1 for _ in range(num_points + 1)]  
    
#     nodes[0] = 0  #X[0] is default 0 to make 1-based indexing
#     return nodes

In [7]:
# select_nodes()

In [8]:
import numpy as np

# function to select destination nodes, for the nodes which want to transmit
def select_destinations(X_vec):
    
    dest = [0 for i in range (num_points + 1)]
    selected_nodes = [i for i, x in enumerate(X_vec) if x == 1]
    
    for node in selected_nodes:
        neighbors = np.array(neighbor_table.loc[f"P{node}"])
        neighbor_indices = np.where(neighbors == 1)[0]
        if len(neighbor_indices) > 0:
            random_neighbor = np.random.choice(neighbor_indices) + 1
            dest[node] = random_neighbor
        else:
            pass
        
    return dest

In [9]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

def get_dist_matrix(x_coords_loc, y_coords_loc):
    
    points = [(x_coords_loc[i], y_coords_loc[i]) for i in range(num_points)]
    distances = pdist(points)
    dist_mat = squareform(distances)
    dist_mat = pd.DataFrame(dist_mat, columns = point_labels, index = point_labels)
    
    return dist_mat

In [10]:
def get_neighbor_table(dist_mat):
    neighbor_tbl = dist_mat
    neighbor_tbl = neighbor_tbl.applymap(lambda x : 1000000 if x > radius else 1)

    for i in range(num_points):
        neighbor_tbl.at[f"P{i + 1}", f"P{i + 1}"] = 0
    
    return neighbor_tbl

In [11]:
def define_c3(dst, neighbor):
    constraint = []
    
    for i in range(len(dst)):
        str = []
        
        if(dst[i] > 0): 
            ind = dst[i]
            
            str.append(ind)   #destination can't transmit also
            
            for j in range(num_points):
                if(neighbor[f"P{ind}"][f"P{j + 1}"] == 1 and j != i - 1):
                    str.append(j + 1)
                    
            constraint.append(str)
            
        else:
            constraint.append('')
        
    return  constraint

In [12]:
import sys
import itertools

def solve_instance(d, c):          #updated the parameters
    
    indexes = [i for i, num in enumerate(d) if num > 0]
    combinations = []
    result = None
    unique_combinations = []
    
      
    for size in range(len(indexes), 0, -1):
        #genetating all combination such that any 1 node finally transmits , or 2 nodes and so on.
        #and these combinations will be checked if they satisfy the constraints
        combinations = itertools.combinations(indexes, size)
        
               
        for combination in combinations:
            valid_comb = True
            for elem in combination:          # EACH element of combination should satisfy the consrtaints
                intersection = set(c[elem]).intersection(set(combination))
                if(len(intersection) > 0):
                    valid_comb = False
                    break
                              
            
            if(valid_comb):                            #for stopping the loop when valid combination found
                result = combination
                break  
            
                       
        if(result != None and len(result) > 0):    #for stopping the loop when valid combination found
            break
                        
    
    return list(result)


In [13]:
import numpy as np
def generate_data(rows):
    
    #data = pd.DataFrame(columns=['Instance', 'Solution'], index=[i for i in range(rows)])
    
    data_X = []
    data_y = []
#     df.loc[0, 'X'] = X
#     df.loc[0, 'Y'] = Y
    
    i = 0
    
    #for i in range(rows):
    while i < rows:
        X = select_nodes()
        
        #print(i, end= " ")
        
        D = select_destinations(X)
        
        if(sum(D) == 0):
            continue
            
        #D = [0, 5, 0, 11, 16, 1, 2, 3, 13, 13, 14, 0, 15, 9, 0, 16, 14, 6, 4, 2, 11]
        
        C3 = define_c3(D, neighbor_table)
        #print(f"C3  {C3}")
        
        sol = solve_instance(D,C3)
        
        
        #print(f" sol is {sol}")
        
        data_X.append(D[1:])
        #data.at[i, 'Instance'] = D[1:]
        
        
        y_var = [0 for i in range (num_points)]
        
        for j in sol:
            y_var[j - 1] = 1
            
        data_y.append(y_var)
        
        #data.at[i, 'Solution'] = sol
        
        i += 1
    
    x_var = np.array(data_X)
    y_var = np.array(data_y)
    
    return x_var, y_var


In [14]:
def accuracy_function(pred_solution, test_solution):
    # Compare each pair of rows
    elementwise_comparison = (pred_solution == test_solution)

    # Count the number of equal elements in each pair
    equal_counts = np.sum(elementwise_comparison, axis=1)

#     print("Number of equal elements in each pair of rows:")
#     print(equal_counts)

    average = np.sum(equal_counts, axis = 0) / (pred_solution.shape[0] * pred_solution.shape[1])
    
    return average
  

In [15]:
def accuracy_function2(pred_solution, test_solution):
    # Compare each pair of rows
    elementwise_comparison = (pred_solution == test_solution)
    
    # Check if all elements in each row are True (i.e., rows are the same)
    rowwise_comparison = np.all(elementwise_comparison, axis=1)


    num_same_rows = np.sum(rowwise_comparison)

    average = num_same_rows / (pred_solution.shape[0])
    
    return average

In [16]:
# Function to generate a new position 4 units away in a random direction
def new_position(x, y):
    dx = random.uniform(-1, 1)  # Random value between -1 and 1
    dy = random.uniform(-1, 1)  # Random value between -1 and 1
    norm = (dx**2 + dy**2)**0.5
    scale = 2 / norm
    new_x = x + scale * dx
    new_y = y + scale * dy
    return new_x, new_y
    
def move_nodes(num_points_to_move): 
#   Randomly select  points to move
    #num_points_to_move = 5
    selected_indices = random.sample(range(len(x_coords)), num_points_to_move)
    print([(item + 1) for item in selected_indices])

    # Update the selected points with new positions
    for index in selected_indices:
        x_new, y_new = new_position(x_coords[index], y_coords[index])
        if(abs(x_new) > 15):
            x_coords[index] = x_coords[index] - (x_new - x_coords[index])
        else:
            x_coords[index] = x_new

        if(abs(y_new) > 15):
            y_coords[index] = y_coords[index] - (y_new - y_coords[index])
        else:
            y_coords[index] = y_new


In [17]:
pd.set_option('display.max_colwidth', None)
np.set_printoptions(threshold=np.inf)

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split

def generate_configurations(num_iterations, num_samples):
    
    prefix_x = "X"
    prefix_y = "y"
    
    for iteration in range(1, num_iterations + 1):
        
        move_nodes(20)
        print(x_coords)
        print(y_coords)
        global dist_matrix
        global neighbor_table
        dist_matrix = get_dist_matrix(x_coords, y_coords)
        neighbor_table = get_neighbor_table(dist_matrix)
        
        global_X = prefix_x + str(iteration)
        global_y = prefix_y + str(iteration)
      
        #saveplot(f"Conf{iteration}.jpg")
        
        globals()[global_X], globals()[global_y] = generate_data(num_samples)
        
        #savedata(globals()[global_X], f"Instance{iteration}.xlsx")
        #savedata(globals()[global_y], f"Solution{iteration}.xlsx")
        
        print(f"Created  configuration {iteration}")
    


In [19]:
# building the classifier.

from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split

def generate_predictions(num_iterations):
    
    prefix_x = "X"
    prefix_y = "y"
    global_test = prefix_x + "test"
    X_test_1 = globals()[global_test]
  
    
    
    for iteration in range(0, num_iterations + 1):
        global_X = prefix_x + str(iteration)
        global_y = prefix_y + str(iteration)
        
        X_set_1  = globals()[global_X]
        y_set_1  = globals()[global_y]
        # Define the classifier
        clf = MultiOutputClassifier(DecisionTreeClassifier(random_state=42, max_depth = 16, min_samples_leaf=4,
                                                   min_samples_split = 16))

        # Train the classifier
        clf.fit(X_set_1, y_set_1)
        
        score = clf.score(X_set_1, y_set_1)
        print('Train Accuracy:', score)
        
        score = clf.score(X_test_1, ytest)
        print('Test Accuracy:', score)
        
        prediction = f"Sol{iteration}"
        globals()[prediction] = clf.predict(X_test_1)
        
        #savedata(globals()[prediction], f"Prediction{iteration}.xlsx")
        print(f"prediction generated {iteration}")
        
        
        
  

In [20]:
def apply_algo(num_iterations):
    
    prefix0 = "Sol0"
    sol_0 = globals()[prefix0]
    
    prefix_a = "sol_alpha"
    
    for iteration in range(1, 10):
        sol_alpha_final =  prefix_a + str(iteration)
        sol_alpha = sol_0
        for i in range(1, num_iterations + 1):
            sol_current = f"Sol{i}"
            selected_array = globals()[sol_current]
            sol_alpha = (iteration * 0.1) * sol_alpha + (1 - (iteration * 0.1)) * selected_array
        print(f"alpha multiplication vector generated {sol_alpha_final}")   
        
        globals()[sol_alpha_final] = sol_alpha
   

In [21]:
dist_matrix = get_dist_matrix(x_coords, y_coords)
neighbor_table = get_neighbor_table(dist_matrix)

#saveplot(f"Conf0.jpg")
X0,y0 = generate_data(10000)
#savedata(X0, f"Instance0.xlsx")
#savedata(y0, f"Solution0.xlsx")

# os.system(f"afplay {notification_sound}")

In [ ]:
generate_configurations(20, 10000)
#os.system(f"afplay {notification_sound}")

Xtest,ytest = generate_data(2000)
#os.system(f"afplay {notification_sound}")
#savedata(Xtest, f"InstanceTest.xlsx")
#savedata(ytest, f"SolutionTest.xlsx")

generate_predictions(20)
apply_algo(20)  

[17, 6, 2, 19, 8, 13, 18, 14, 16, 12, 3, 9, 20, 1, 15, 10, 7, 5, 11, 4]
[-3.4790663965287445, -8.713999481345725, 0.6508074219667108, 4.075285706113471, 3.9711376353722048, 13.006283053444347, -12.373397487771609, -12.711692056111282, 3.9606100534712603, 4.692409043807181, 8.4508080629422, 8.189819056699434, -3.462917542850715, -11.263185159045904, -7.143674647858506, -5.799832221829793, -3.1184183515867665, -8.31936578548992, 5.508434369503189, 11.991112959794549]
[-13.522869940424474, 3.680279086763969, 3.2285739251481207, -3.715642817740255, 5.991903213763344, 10.401321482931772, -11.553796276840586, -4.877578258932768, -7.821529273948939, -13.008476791847542, -5.976782262339441, 14.95579970026822, -4.625855917647781, 4.183340090354285, 5.021712053032829, -10.866612750182414, 7.007183187824004, -12.371953090075976, 2.3533536771573376, 8.433589084867894]


In [ ]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.model_selection import train_test_split

# X_set_1  = X0
# y_set_1  = y0
# # Define the classifier
# clf = MultiOutputClassifier(DecisionTreeClassifier(random_state=42, max_depth = 16, min_samples_leaf=4,
#                                            min_samples_split = 16))

# # Train the classifier
# clf.fit(X_set_1, y_set_1)

# score = clf.score(X_set_1, y_set_1)
# print('Train Accuracy:', score)

# score = clf.score(Xtest, ytest)
# print('Test Accuracy:', score)

# prediction = clf.predict(Xtest)

# accuracy_function(prediction, ytest)

In [ ]:
# savedata(prediction, "Prediction0.xlsx")

In [ ]:
print(accuracy_function(Sol1, ytest))
print(accuracy_function(Sol4, ytest))
print(accuracy_function(Sol8, ytest))
print(accuracy_function(Sol12, ytest))
print(accuracy_function(Sol16, ytest))
print(accuracy_function(Sol20, ytest))
print(accuracy_function(Sol25, ytest))
print(accuracy_function(Sol30, ytest))
print(accuracy_function(Sol36, ytest))
print(accuracy_function(Sol40, ytest))


print(accuracy_function2(Sol1, ytest))
print(accuracy_function2(Sol4, ytest))
print(accuracy_function2(Sol8, ytest))
print(accuracy_function2(Sol12, ytest))
print(accuracy_function2(Sol16, ytest))
print(accuracy_function2(Sol20, ytest))
print(accuracy_function2(Sol25, ytest))
print(accuracy_function2(Sol30, ytest))
print(accuracy_function2(Sol36, ytest))
print(accuracy_function2(Sol40, ytest))

In [ ]:
# # checking if value is greater than threshold 0.5
# y_pred_3_1 = np.where(y_pred_3_1 > 0.5, 1, 0)
# y_pred_3_2 = np.where(y_pred_3_2 > 0.5, 1, 0)
# y_pred_3_3 = np.where(y_pred_3_3 > 0.5, 1, 0)
# y_pred_3_4 = np.where(y_pred_3_4 > 0.5, 1, 0)
# y_pred_3_5 = np.where(y_pred_3_5 > 0.5, 1, 0)
# y_pred_3_6 = np.where(y_pred_3_6 > 0.5, 1, 0)
# y_pred_3_7 = np.where(y_pred_3_7 > 0.5, 1, 0)
# y_pred_3_8 = np.where(y_pred_3_8 > 0.5, 1, 0)
# y_pred_3_9 = np.where(y_pred_3_9 > 0.5, 1, 0)

# checking if value is greater than threshold 0.5
sol_alpha1 = np.where(sol_alpha1 > 0.5, 1, 0)
sol_alpha2 = np.where(sol_alpha2 > 0.5, 1, 0)
sol_alpha3 = np.where(sol_alpha3 > 0.5, 1, 0)
sol_alpha4 = np.where(sol_alpha4 > 0.5, 1, 0)
sol_alpha5 = np.where(sol_alpha5 > 0.5, 1, 0)
sol_alpha6 = np.where(sol_alpha6 > 0.5, 1, 0)
sol_alpha7 = np.where(sol_alpha7 > 0.5, 1, 0)
sol_alpha8 = np.where(sol_alpha8 > 0.5, 1, 0)
sol_alpha9 = np.where(sol_alpha9 > 0.5, 1, 0)

In [ ]:
accuracy_vec = []
accuracy_vec.append( accuracy_function(sol_alpha1, ytest))
accuracy_vec.append( accuracy_function(sol_alpha2, ytest))
accuracy_vec.append( accuracy_function(sol_alpha3, ytest))
accuracy_vec.append( accuracy_function(sol_alpha4, ytest))
accuracy_vec.append( accuracy_function(sol_alpha5, ytest))
accuracy_vec.append( accuracy_function(sol_alpha6, ytest))
accuracy_vec.append( accuracy_function(sol_alpha7, ytest))
accuracy_vec.append( accuracy_function(sol_alpha8, ytest))
accuracy_vec.append( accuracy_function(sol_alpha9, ytest))
accuracy_vec

# accuracy_vec = []
# accuracy_vec.append( accuracy_function2(sol_alpha1, ytest))
# accuracy_vec.append( accuracy_function2(sol_alpha2, ytest))
# accuracy_vec.append( accuracy_function2(sol_alpha3, ytest))
# accuracy_vec.append( accuracy_function2(sol_alpha4, ytest))
# accuracy_vec.append( accuracy_function2(sol_alpha5, ytest))
# accuracy_vec.append( accuracy_function2(sol_alpha6, ytest))
# accuracy_vec.append( accuracy_function2(sol_alpha7, ytest))
# accuracy_vec.append( accuracy_function2(sol_alpha8, ytest))
# accuracy_vec.append( accuracy_function2(sol_alpha9, ytest))
# accuracy_vec

In [ ]:
import matplotlib.pyplot as plt

alpha_vec = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,  0.9]

plt.plot(alpha_vec, accuracy_vec, marker='o')

# Adding labels and title
plt.xlabel('alpha')
plt.ylabel('Test Accuracy')
plt.title('Test Accuracy vs. alpha values')
plt.savefig("plot.jpg")
# Display the plot
plt.show()

In [ ]:
# import pandas as pd

# opt_solution = pd.read_excel('/Users/rob/Desktop/Semester data/Sem 4 S23/CS6340 Prof Haas/Volunteer Project/Incremental Learning/experiments/1data/experiment_test/SolutionTest.xlsx')


# pred = pd.read_excel('/Users/rob/Desktop/Semester data/Sem 4 S23/CS6340 Prof Haas/Volunteer Project/Incremental Learning/experiments/1data/experiment_test/Prediction0.xlsx')


# elementwise_comparison = (pred == opt_solution)


# equal_counts = np.sum(elementwise_comparison, axis=1)

# equal_counts

# df = pd.concat([opt_solution, equal_counts], axis=1)

# df.to_excel("counts.xlsx", index=False)
os.system(f"afplay {notification_sound}")